In [37]:
from pyramid.arima import auto_arima
import pandas as pd
import numpy as np

In [38]:
data_train = pd.read_csv('/home/affine/Downloads/train.csv')
data_test = pd.read_csv('/home/affine/Downloads/test.csv')

In [39]:
data_test.shape

(33, 389)

In [40]:
data_train.columns

Index(['Unnamed: 0', 'High1222.HK', 'Low1222.HK', 'Open1222.HK',
       'Close1222.HK', 'High4715.KL', 'Low4715.KL', 'Open4715.KL',
       'Close4715.KL', 'High5398.KL',
       ...
       'OpenYG2.BE', 'CloseYG2.BE', 'HighZARFF', 'LowZARFF', 'OpenZARFF',
       'CloseZARFF', 'High^IXIC', 'Low^IXIC', 'Open^IXIC', 'Close^IXIC'],
      dtype='object', length=389)

In [41]:
train = data_train['Close^IXIC']
test = data_test['Close^IXIC']

In [42]:
print(train.isna().sum())
print(test.isna().sum())

0
0


In [43]:
stepwise_model = auto_arima(train, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1107.688, BIC=1120.017, Fit time=1.252 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1139.881, BIC=1144.813, Fit time=0.024 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=1118.871, BIC=1128.735, Fit time=0.639 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1106.596, BIC=1116.460, Fit time=0.767 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=1108.594, BIC=1120.923, Fit time=1.373 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=1138.113, BIC=1145.510, Fit time=0.129 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=1106.955, BIC=1114.353, Fit time=0.369 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 1, 1, 

In [26]:
print(stepwise_model.aic())

1106.5964749713373


In [27]:
stepwise_model.fit(train)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 1, 1, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [28]:
future_forecast = stepwise_model.predict(n_periods=33)

In [29]:
type(test)

pandas.core.series.Series

In [30]:
test_data = pd.DataFrame({'actual': test, 'predicted': future_forecast})

In [31]:
test_data

actual    predicted
0   5029.431470  5052.301293
1   5073.041837  5107.110414
2   5082.809970  5211.498964
3   4934.622861  5267.774185
4   4748.004325  5372.017705
5   4879.036821  5398.660266
6   5082.508032  5477.239066
7   5040.540039  5577.958211
8   4610.714253  5658.658444
9   4463.206030  5762.272666
10  4754.479115  5851.997744
11  4892.168108  5949.544197
12  4788.235235  6034.125551
13  4856.232688  6100.871889
14  5023.988965  6217.197655
15  5217.038319  6285.410092
16  5254.154297  6401.590828
17  5255.989560  6440.170606
18  5260.565244  6530.686622
19  5413.116653  6643.342983
20  5561.423486  6735.980432
21  5764.702097  6851.531871
22  5864.964334  6953.194165
23  5912.169485  7062.677834
24  6124.878174  7159.196405
25  6224.285889  7237.879958
26  6294.674023  7366.142941
27  6311.397397  7446.292595
28  6428.078979  7574.410546
29  6603.011341  7624.927541
30  6796.045224  7727.380773
31  6889.740454  7851.974350
32  6889.740454  7956.549016

In [33]:
test_data['APE'] = abs(test_data['actual'] - test_data['predicted'])/test_data['actual']

In [36]:
print('Mape is :', np.mean(test_data['APE']))

Mape is : 0.16936592849546167
